In [11]:
!pip install imbalanced-learn==0.11.0


In [15]:
# ============================================================
# SCRIPT 1: segmentation_creator.py
# Creates labeled dataset (critical / habitual / occasional)
# ============================================================

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# ------------------------------------------------------------
# 1. LOAD ALL 5 TABLES FROM IBM TELCO DATA
# ------------------------------------------------------------
base_path = "/kaggle/input/hcldata"

demo = pd.read_csv(f"{base_path}/Telco_customer_churn_demographics.csv")
loc  = pd.read_csv(f"{base_path}/Telco_customer_churn_location.csv")
pop  = pd.read_csv(f"{base_path}/Telco_customer_churn_population.csv")
serv = pd.read_csv(f"{base_path}/Telco_customer_churn_services.csv")
stat = pd.read_csv(f"{base_path}/Telco_customer_churn_status.csv")

print("Loaded Shapes:")
print("Demographics:", demo.shape)
print("Location    :", loc.shape)
print("Population  :", pop.shape)
print("Services    :", serv.shape)
print("Status      :", stat.shape)

# ------------------------------------------------------------
# 2. MERGE TABLES ON CUSTOMER ID
# ------------------------------------------------------------
df = (
    serv.merge(stat, on="Customer ID", suffixes=("_serv", "_status"))
        .merge(demo, on="Customer ID")
        .merge(loc, on="Customer ID", suffixes=("_demo", "_loc"))
)

df = df.merge(pop, on="Zip Code", how="left")

print("\nMerged Shape:", df.shape)

# ------------------------------------------------------------
# 3. CLEANING
# ------------------------------------------------------------
# Clean object columns
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype(str).str.strip()

# Numeric conversions
numeric_candidates = [
    "Tenure in Months","Avg Monthly Long Distance Charges",
    "Avg Monthly GB Download","Monthly Charge","Total Charges",
    "Total Refunds","Total Extra Data Charges",
    "Total Long Distance Charges","Churn Score","CLTV",
    "Population","Number of Dependents","Number of Referrals"
]

for col in numeric_candidates:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill numeric NaN
num_cols = df.select_dtypes(include=["int64","float64"]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

print("\nClean Columns:", len(df.columns))

# ------------------------------------------------------------
# 4. RULE-BASED SEGMENTATION
# ------------------------------------------------------------
df_seg = df.copy()

df_seg["Churn Value"] = df_seg.get("Churn Value", 0)
df_seg["Churn Score"] = df_seg.get("Churn Score", 0)

# Quantiles
total_q60   = df_seg["Total Charges"].quantile(0.60)
cltv_q60    = df_seg["CLTV"].quantile(0.60)
churn_q50   = df_seg["Churn Score"].quantile(0.50)
churn_q70   = df_seg["Churn Score"].quantile(0.70)
refund_q60  = df_seg["Total Refunds"].quantile(0.60)
extra_q60   = df_seg["Total Extra Data Charges"].quantile(0.60)

segments = []

for _, r in df_seg.iterrows():

    # -------------------------------
    # 1. CRITICAL CUSTOMER RULES
    # -------------------------------
    family_like = (
        (r["Dependents"] == "Yes") or
        (r["Multiple Lines"] == "Yes") or
        (r["Number of Dependents"] > 0)
    )

    high_value = (
        (r["Total Charges"] >= total_q60) or
        (r["CLTV"] >= cltv_q60)
    )

    low_risk = (r["Churn Score"] <= churn_q50)

    if sum([family_like, high_value, low_risk]) >= 2:
        segments.append("critical")
        continue

    # -------------------------------
    # 2. HABITUAL DEFAULTER
    # -------------------------------
    high_churn = (r["Churn Score"] >= churn_q70) or (r["Churn Value"] == 1)
    financial_instability = (
        (r["Total Refunds"] >= refund_q60) or
        (r["Total Extra Data Charges"] >= extra_q60)
    )

    if high_churn and financial_instability:
        segments.append("habitual_defaulter")
        continue

    # -------------------------------
    # 3. OCCASIONAL DEFAULTER
    # -------------------------------
    segments.append("occasional_defaulter")

df_seg["segment"] = segments

print("\nFinal Segmentation Distribution:")
print(df_seg["segment"].value_counts())

# ------------------------------------------------------------
# 5. SAVE FINAL LABELED DATASET
# ------------------------------------------------------------
output_path = "/kaggle/working/final.csv"
df_seg.to_csv(output_path, index=False)

print("\nSaved labeled dataset to:")
print(output_path)


Loaded Shapes:
Demographics: (7043, 9)
Location    : (7043, 10)
Population  : (1671, 3)
Services    : (7043, 31)
Status      : (7043, 12)

Merged Shape: (7043, 61)

Clean Columns: 61

Final Segmentation Distribution:
segment
critical                4058
habitual_defaulter      1716
occasional_defaulter    1269
Name: count, dtype: int64

Saved labeled dataset to:
/kaggle/working/final.csv
